In [1]:
import torchvision.datasets as datasets
import torchvision.models as models
import torch
from torchvision import transforms
from keras.utils import to_categorical
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

2023-05-14 19:41:40.962390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 19:41:41.166469: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-14 19:41:41.853688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: usr/local/cuda-11.8/lib64
2023-05-14 19:41:41.853865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

Config

In [2]:
config = {
    'batch_size':128,
    'learning_rate':1e-5,
    'weight_decay':5e-4,
    'num_epochs':20
}

Load Datasets

In [3]:
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)

train_transform = [transforms.Resize((224, 224)),
                   transforms.RandomHorizontalFlip(p=0.5),
                   transforms.RandomVerticalFlip(p=0.5),
                   transforms.ToTensor(),
                   transforms.Normalize(mean, std),
                  ]
train_transform = transforms.Compose(train_transform)

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])

In [4]:
cifar100_trainset = datasets.CIFAR100(root='/home/user01/cifar100/train/', train=True,
                                    download=False, transform=train_transform)
trainloader = torch.utils.data.DataLoader(
    cifar100_trainset, batch_size=config['batch_size'], shuffle=True, num_workers=7)

In [5]:
cifar100_testset = datasets.CIFAR100(root='/home/user01/cifar100/test/', train=False,
                                   download=False, transform=test_transform)
testloader = torch.utils.data.DataLoader(
    cifar100_testset, batch_size=config['batch_size'], shuffle=False, num_workers=7)

Model Definition

In [6]:
# get_graph_node_names(model)

In [7]:
new_model = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')

In [8]:
# for param in new_model.parameters():
#     param.requires_grad = False

In [9]:
new_model.fc = torch.nn.Linear(in_features=512, out_features=100, bias=True)

In [10]:
# inp = np.random.rand(2,3,32,32)
# new_model(torch.Tensor(inp[:2]))

# Finetuning with CIFAR10

In [11]:
def label_to_out(label):
    out = np.zeros(100)
    out[label] = 1
    return out

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [13]:
new_model = new_model.to(device)

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()

In [15]:
for name, param in new_model.named_parameters():
    print(f'{name}')

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer2.2.conv1.weight
layer2.2.bn1.weight
layer2.2.bn1.bias
layer2.2.conv2.weight
layer2.2.bn2.weight
layer2.2.bn2.bias
layer2.3.conv1.weight
layer2.3.bn1.weight
layer2.3.bn1.bias
layer2.3.conv2.weight
layer2.3.bn2.weight
layer2.3.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight


In [16]:
params_1x = [param for name, param in new_model.named_parameters() if 'fc' not in str(name)]
optimizer = torch.optim.Adam([{'params':params_1x}, {'params': new_model.fc.parameters(), 'lr': config['learning_rate']*10}],
                             lr=config['learning_rate'], weight_decay=config['weight_decay'])

In [17]:
def output_to_label(out):
    return out.argmax()

In [18]:
# for batch, (features, labels) in enumerate(trainloader):
#     if batch == 0:
#         out = new_model(features.to(device))
#         print(out.shape)
#         print(features.shape)
#         print(labels.shape)
#         print(loss_fn(out, labels.to(device)))

In [19]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        features, labels = features.to(device), labels.to(device)
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]\n")


def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            features, labels = features.to(device), labels.to(device)
            pred = model(features)
            if batch == 0:
                global tmp_features
                global tmp_labels
                global tmp_pred
                tmp_features = features
                tmp_labels = labels
                tmp_pred = pred
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.argmax(axis=1).cpu().numpy())
            # convert the original labels corresponding to the current batch to a numpy array
            true_lst = labels
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==t else 0 for (p, t) in zip(pred_lst, true_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    print(f"Epoch {epoch_num} - {name} Accuracy: {correct*100}%, Avg loss: {sum_test_loss}\n")

In [20]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(trainloader, new_model, loss_fn, optimizer, epoch_num)
    test_loop(testloader, new_model, loss_fn, epoch_num, 'Test')

Epoch 1 - loss: 4.762279  [    0/50000]

Epoch 1 - loss: 4.057532  [12800/50000]

Epoch 1 - loss: 3.181620  [25600/50000]

Epoch 1 - loss: 2.471166  [38400/50000]

Epoch 1 - Test Accuracy: 53.010000000000005%, Avg loss: 0.016136525762081147

Epoch 2 - loss: 2.079953  [    0/50000]

Epoch 2 - loss: 1.693331  [12800/50000]

Epoch 2 - loss: 1.651588  [25600/50000]

Epoch 2 - loss: 1.542446  [38400/50000]

Epoch 2 - Test Accuracy: 65.60000000000001%, Avg loss: 0.010418665897846222

Epoch 3 - loss: 1.528598  [    0/50000]

Epoch 3 - Test Accuracy: 70.27%, Avg loss: 0.008427358275651931

Epoch 4 - loss: 1.282648  [    0/50000]

Epoch 4 - loss: 1.029013  [12800/50000]

Epoch 4 - loss: 0.970660  [25600/50000]

Epoch 4 - loss: 1.121636  [38400/50000]

Epoch 4 - Test Accuracy: 72.97%, Avg loss: 0.007434784436225891

Epoch 5 - loss: 0.964223  [    0/50000]

Epoch 5 - loss: 1.096099  [12800/50000]

Epoch 5 - loss: 0.914262  [25600/50000]

Epoch 5 - loss: 0.869689  [38400/50000]

Epoch 5 - Test Acc

In [ ]:
torch.save(new_model, 'cifar100.pth')

In [21]:
# test_loop(testloader, new_model, loss_fn, epoch_num, 'Test')